In [1]:
%load_ext autoreload
%autoreload 2

In [42]:
import stock_automation_utils as SAU
import backtest_utils as BU
import pickle
from datetime import datetime, timedelta
import numpy as np

In [3]:
stock_symbols_pickle = 'stock_symbols.pkl'  # Name of the pickle file

In [4]:
# Load symbols into the stock_symbols.pkl
symbols = []  # Example list of stock symbols
if len(symbols) > 0:
    SAU.add_stock_symbols_to_pickle(symbols, stock_symbols_pickle)

In [5]:
# Load the data from 'stock_symbols.pkl' into a variable called 'symbols'
with open(stock_symbols_pickle, 'rb') as file:
    symbols = pickle.load(file)

## BACKTEST

In [6]:
# First load the data once. (from start_date-380 to end_date + (rebalance frq * 2)
# For each rebalance period, run the analysis and get long short stocks.
# Decide how much to invest in each stock and calculate qty int
# now find out the buy price and stoploss (from the risk%)
# now for each stock find out if in the test period if the stoploss is hit. if yes, book a loss.
# for stocks that stoploss not hit, calculate the closing P/L for each rebalance freq. (print it)
####### from prices_array, 
# In the end, sum up all the P/L.
####### profits = sum(rebalance_periods_profit)

In [99]:
def trading_simulation(portfolio_value, trading_symbols_interval, trading_data_interval, risk_pct):
    def calculate_position_size(portfolio_value, trading_symbols_interval):
        position_size = (np.array(range(len(trading_symbols_interval))) + 1)
        position_size = portfolio_value/position_size
        position_size = (position_size / np.sum(position_size)) * portfolio_value
        position_size = position_size/sum(position_size)
        position_size = (np.sqrt(position_size) + position_size)/2
        position_size = position_size/sum(position_size)
        position_size = position_size * portfolio_value
        
        return position_size
    def get_buy_price(trading_data_interval):
        buy_price = []
        for symbol in trading_data_interval:
            buy_price.append(trading_data_interval[symbol]['Adj Close'].iloc[0])
        return np.array(buy_price)
    def get_sell_price(trading_data_interval, stoploss):
        sell_price = []
        for count, symbol in enumerate(trading_data_interval):
            if trading_data_interval[symbol]['Adj Close'].min() < stoploss[count]:
                sell_price.append(stoploss[count])
            else:
                sell_price.append(trading_data_interval[symbol]['Adj Close'].iloc[-1])
        return np.array(sell_price)
    
    position_size = calculate_position_size(portfolio_value, trading_symbols_interval)
    buy_price = get_buy_price(trading_data_interval)
    qty = np.array(position_size/buy_price, dtype=int)
    stoploss = buy_price*(1-risk_pct)
    sell_price = get_sell_price(trading_data_interval, stoploss)
    profit = (sell_price - buy_price)*qty
    trading_simulation_array = np.array([trading_symbols_interval, buy_price, qty, stoploss, sell_price, profit])
    profit_interval = np.sum(profit)
    
    return trading_simulation_array, profit_interval

In [100]:
start_date = '2022-01-01'
end_date = '2022-12-31'
start_date_dt = datetime.strptime(start_date, "%Y-%m-%d")
end_date_dt = datetime.strptime(end_date, "%Y-%m-%d")
rebalance_frequency = 15  # In days
# create a numpy array of numbers 15 days apart from 0 to 365
days_ago_list = list(range(0, 365, rebalance_frequency))
long_count = 10
short_count = 0
portfolio_value = 50000
risk_pct = 0.10

In [101]:
# For each rxebalance period, run the analysis and get long short stocks.
rebalance_periods = BU.calculate_rebalance_periods_with_dates(start_date_dt, end_date_dt, rebalance_frequency)
rebalance_periods = [(datetime.combine(rebalance_periods[0], datetime.min.time()), datetime.combine(rebalance_periods[1], datetime.min.time())) for rebalance_periods in rebalance_periods]
full_stock_data = SAU.load_all_stock_data(symbols, start_date_dt-timedelta(days=365+(rebalance_frequency*2)), end_date_dt+timedelta(days=rebalance_frequency*2))
symbols = list(full_stock_data.keys())

In [103]:
backtest_runs = []
backtest_profits = []
for rebal_internal in rebalance_periods:
    start_date_interval = rebal_internal[0]
    end_date_interval = rebal_internal[1]
    # Run the test for one interval
    # Load all stock data
    historical_data_interval = SAU.get_data_for_interval(symbols, full_stock_data, start_date_interval-timedelta(days=365), start_date_interval-timedelta(days=1))

    # Get the analysis array
    symbols_interval, data_analysis_interval, data_index_interval, dates_interval = SAU.get_analysis_array(symbols, start_date_interval, historical_data_interval, days_ago_list)
    # Get long and short stocks
    long_symbols, short_symbols, symbols_array, data_array, data_index = BU.get_long_short_symbols(long_count, short_count, symbols_interval, data_analysis_interval, data_index_interval)
    # Get the trading symbols
    trading_symbols_interval = list(long_symbols) + list(short_symbols)
    # Get the trading data for the interval
    trading_data_interval = SAU.get_data_for_interval(trading_symbols_interval, full_stock_data, start_date_interval, end_date_interval)
    # Run the trades
    trading_simulation_array, profit_interval = trading_simulation(portfolio_value, trading_symbols_interval, trading_data_interval, risk_pct)
    backtest_runs.append(trading_simulation_array)
    backtest_profits.append(profit_interval)
    print({f"Profit for the interval {start_date_interval} to {end_date_interval}": profit_interval})

{'Profit for the interval 2022-01-01 00:00:00 to 2022-01-15 00:00:00': -2763.280346298218}
{'Profit for the interval 2022-01-16 00:00:00 to 2022-01-30 00:00:00': 403.7163721084589}
{'Profit for the interval 2022-01-31 00:00:00 to 2022-02-14 00:00:00': 1946.5412015914917}
{'Profit for the interval 2022-02-15 00:00:00 to 2022-03-01 00:00:00': 3241.3845567703233}
{'Profit for the interval 2022-03-02 00:00:00 to 2022-03-16 00:00:00': -2300.020388126373}
{'Profit for the interval 2022-03-17 00:00:00 to 2022-03-31 00:00:00': 308.0416267395044}
{'Profit for the interval 2022-04-01 00:00:00 to 2022-04-15 00:00:00': 1919.5421090126038}
{'Profit for the interval 2022-04-16 00:00:00 to 2022-04-30 00:00:00': -3223.736762809753}
{'Profit for the interval 2022-05-01 00:00:00 to 2022-05-15 00:00:00': 1122.6821079254146}
{'Profit for the interval 2022-05-16 00:00:00 to 2022-05-30 00:00:00': 3387.5600414276155}
{'Profit for the interval 2022-05-31 00:00:00 to 2022-06-14 00:00:00': -1868.5465927124048}


In [106]:
sum(backtest_profits)

-8110.134854698169

# Update data for all stocks.
SAU.load_and_update_stock_data()

#### Load up the Initial data for starting analysis

In [ ]:
# symbols = ['META', 'NVDA', 'SYM', 'CRWD', 'AMD', 'URI']  # List of symbols
start_date = "2024-02-04"  # Start date for analysis
symbols_array, data_array, data_index = SAU.get_data_array_symbols(symbols, start_date)

#### Run the analysis

In [ ]:
long_count = 10
short_count = 10
long_symbols, short_symbols, symbols_array, data_array, data_index = BU.get_long_short_symbols(long_count, short_count, symbols_array, data_array, data_index)

#### BACKTEST

In [ ]:
start_date = '2022-01-01'
end_date = '2022-12-31'
rebalance_frequency = 15  # In days

In [ ]:
# First load the data once. (from start_date-380 to end_date + (rebalance frq * 2)
# For each rebalance period, run the analysis and get long short stocks.
# Decide how much to invest in each stock and calculate qty int
# now find out the buy price and stoploss (from the risk%)
# now for each stock find out if in the test period if the stoploss is hit. if yes, book a loss.
# for stocks that stoploss not hit, calculate the closing P/L for each rebalance freq. (print it)
# In the end, sum up all the P/L.


In [ ]:
''' 
TO DO: 
- Start creating the backtest
- When the backtest is running, integrate with IBKR. Start with getting positions + cash.
- Then calculate ideal portfolio, keeping in mind that you only want to use 75% of your total available margin.
- Then figure out ideal vs current portfolio and calculate the orders that need to be sent.
- Then figure out how to sent orders, with a stoploss built in.


'''

# BACKTESTING

In [ ]:
# Example usage
pickle_file = 'stock_symbols.pkl'
start_date = '2022-01-01'
end_date = '2022-12-31'
rebalance_frequency = 15  # In days

rebalance_periods = BU.calculate_rebalance_periods_with_dates(pickle_file, start_date, end_date, rebalance_frequency)
for period in rebalance_periods:
    print(f"Rebalance Period: Start Date = {period[0]}, End Date = {period[1]}")

In [ ]:
rebalance_periods